### Hyperparameters Search for GCN Model

- Prediction of aqueous solubility of small organic molecules using:
    - Graph Convolutional Neural Networks (GCNs)
    
- Finding the best hyperparams for N=20 epochs, then train the best model.
- Weight and Biases for logging

In [24]:
# libraries
import os, math
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch_geometric.data import DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# pytorch
import torch
import torch.nn.functional as F 
from torch.nn import Linear
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool as gap
from torch_geometric.nn import global_max_pool as gmp

# from local files
from custom_dataset import MoleculeDataset
from model import GCN, GAT

import wandb

In [25]:
class GCN(torch.nn.Module):
    def __init__(self, n_features, hidden_channels, dropout_p=0.4):
        super(GCN, self).__init__()
        torch.manual_seed(21)
        self.conv1 = GCNConv(n_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels))
        self.conv3 = GCNConv(int(hidden_channels), int(hidden_channels))
        self.linear = Linear(int(hidden_channels), 1)
        self.dropout_p = dropout_p

    def forward(self, data, edge_index, batch):
        x, targets = data.x, data.y
        # 1. Obtain the node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        
        # 2. Aggregating message passing/embeddings
        x = gap(x, batch)
        
        # 3. Apply the final classifier
        x = F.dropout(x, p=self.dropout_p, training=self.training)

        # model output from forward and loss 
        out = self.linear(x)  
        loss = torch.nn.BCEWithLogitsLoss()(out, targets.reshape(-1, 1).type_as(out))

        out = torch.sigmoid(out) # converting out proba in range [0, 1]
        return out, loss

In [26]:
# Constants
batch_size=64

In [27]:
# model from model.py
model = GCN(n_features=30, hidden_channels=128)
print(model)

GCN(
  (conv1): GCNConv(30, 128)
  (conv2): GCNConv(128, 128)
  (conv3): GCNConv(128, 128)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)


In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cpu


In [29]:
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [35]:
wandb.login(key="")

In [36]:
wandb.init(project="", entity="")

In [10]:
# training and prediction with the model
class Trainer:
    
    def __init__(self, model, optimizer, train_loader, valid_loader, batch_size=64):
        self.model = model
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.batch_size = batch_size
        self.optimizer = optimizer

    # training model
    def train_one_epoch(self, epoch):
        self.model.train()

        t_targets = []; p_targets = []; losses = []
        tqdm_iter = tqdm(self.train_loader, total=len(self.train_loader))
        for i, data in enumerate(tqdm_iter):

            tqdm_iter.set_description(f"Epoch {epoch}")
            self.optimizer.zero_grad()
            outputs, loss = self.model(data, data.edge_index, data.batch)
            targets = data.y
            loss.backward()
            self.optimizer.step()

            y_true = self.process_output(targets)  # for one batch
            y_proba = self.process_output(outputs.flatten()) # for one batch

            auc = roc_auc_score(y_true, y_proba)
            tqdm_iter.set_postfix(train_loss=loss.item(), train_auc=auc, valid_loss=None, valid_auc=None)
            
            losses.append(loss.item())
            t_targets.extend(list(y_true))
            p_targets.extend(list(y_proba))

        epoch_auc = roc_auc_score(t_targets, p_targets)
        epoch_loss = sum(losses)/len(losses)
        return epoch_loss, epoch_auc, tqdm_iter


    def process_output(self, out):
        out = out.cpu().detach().numpy()
        return out

    
    def validate_one_epoch(self, progress=None):
        # model in eval model
        self.model.eval()
        
        t_targets = []; p_targets = []; losses = []
        for data in self.valid_loader:
            
            outputs, loss = self.model(data, data.edge_index, data.batch)
            outputs, targets = outputs.flatten(), data.y
            
            y_proba = self.process_output(outputs)  # for one batch
            y_true = self.process_output(targets) # for one batch 
            
            t_targets.extend(list(y_true))
            p_targets.extend(list(y_proba))
            losses.append(loss.item())
            
        # progress stats
        epoch_auc = roc_auc_score(t_targets, p_targets)
        epoch_loss = sum(losses)/len(losses)
        if progress:
            progress_tracker = progress["tracker"]
            train_loss = progress["loss"]
            train_auc = progress["auc"]
            progress_tracker.set_postfix(train_loss=train_loss, train_auc=train_auc, valid_loss=epoch_loss, valid_auc=epoch_auc)              
            progress_tracker.close()
        return epoch_loss, epoch_auc
            
    # runs the training and validation trainer for n_epochs
    def run(self, n_epochs=10):
        
        train_scores = []; train_losses = []
        valid_scores = []; valid_losses = []
        for e in range(1, n_epochs+1):
            lt, at, progress_tracker = self.train_one_epoch(e)
            
            train_losses.append(lt)
            train_scores.append(at)
            
            # validate this epoch
            progress = {"tracker": progress_tracker, "loss": lt, "auc": at}  
            lv, av = self.validate_one_epoch(progress=progress)  # pass training progress tracker to validation func
            valid_losses.append(lv)
            valid_scores.append(av)
        
        return (train_losses, train_scores), (valid_losses, valid_scores)
            
        
    def predict(self, test_loader):
        
        self.model.eval()
        predictions = []
        tqdm_iter = tqdm(test_loader, total=len(test_loader))
        for data in tqdm_iter:
            with torch.no_grad():
                o, _ = self.model(data, data.edge_index, data.batch)
                o = self.process_output(o.flatten())
                predictions.extend(list(o))
            tqdm_iter.set_postfix(stage="test predictions")
        tqdm_iter.close()
        return np.array(predictions)

#### Dataset: Custom molecular dataset (created in custom_dataset.py)

In [33]:
train_dataset = MoleculeDataset(root="Dataset/", filename="solubility-dataset-train.csv")
test_dataset = MoleculeDataset(root="Dataset/", filename="solubility-dataset-test.csv", test=True)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
# data loaders size
train_loader.dataset.length, test_loader.dataset.length

(8847, 986)

In [34]:
sweep_config = {
                  "name" : "my-sweep",
                  "method" : "bayes",
                  "metric": {"name": "AUC", "goal": "maximize"},
                  "parameters" : {
                    "lr": { "min": 0.01, "max": 0.1},
                    "weight_decay": {"min": 1e-5, "max": 1e-2},
                    "hidden_channels": {"values": [32, 64, 128, 256] },
                    "dropout_p": {"values": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]},
                    "epochs": {"values": [20]},  # training each models for only 20 epochs 
                  }
            }

sweep_id = wandb.sweep(sweep_config)

In [21]:
def get_true_test_labels(test_loader):
    y_true_list = []
    for data in test_loader:
        ys = data.y.flatten()
        y_true_list.extend(list(ys.cpu().detach().numpy()))
    return np.array(y_true_list)

In [22]:
def tune_hyperparams(config=None):
    N_EPOCH = 10
    with wandb.init(config=config):
        config = wandb.config
        
        # model prep and training
        model = GCN(n_features=30, hidden_channels=config.hidden_channels, dropout_p=config.dropout_p)
        model.to(device)
        # Optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
        trainer = Trainer(model=model, optimizer=optimizer, train_loader=train_loader, valid_loader=test_loader)

        (train_losses, train_scores), (valid_losses, valid_scores) = trainer.run(n_epochs=config.epochs)
        
        # logging hyperparameters
        params = {}
        for p, v in config.items():
            params[p] = v
        wandb.log(params)
        
        y_proba = trainer.predict(test_loader)
        y_true = get_true_test_labels(test_loader=test_loader)
        
        # AUC on validation dataset
        auc = roc_auc_score(y_true, y_proba)
        wandb.log({"AUC": auc})

In [37]:
wandb.agent(sweep_id, function=tune_hyperparams, count=50, project="")

## Results from 50 Bayes search runs
- Each run is trained only for 20 epochs.
- Next, further train the best hyper-params combination for more epoches.


![alt text](params_tuning.PNG "Results from the 50 different searches")

- Bayes hyper-params tuner in WandB seems working! Only a few runs with very relatively small AUC at the beginning and most of the remaining runs towards the upper range of the AUC output.

- Will train the best model (from the hyperparams combination above) for best possible performance in training notebook.